# Solving Sudoku Puzzles
The goal of this notebook is to walk through the process of building a Sudoku solver that can handle puzzles of varying difficulty. The approach relies on classical AI methods, particularly backtracking search with constraint checking, which are common techniques for solving constraint satisfaction problems.


## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

Throughout the notebook:

We load and explore Sudoku datasets of multiple difficulty levels.

We implement a robust sudoku_solver() function that can solve valid puzzles and correctly reject invalid ones.

We include performance checks and automated test cases to evaluate the effectiveness and correctness of the implementation.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One – Implementing the Sudoku Solver
In this section, we will build the core functionality of our Sudoku-solving agent.

You are expected to implement a function named sudoku_solver(sudoku) that takes a single Sudoku puzzle as input – represented as a 9x9 NumPy array – and returns the solved puzzle as another 9x9 NumPy array.

Empty cells in the puzzle are represented by 0s.

If the puzzle has no valid solution, the function should return a 9x9 array filled with -1s.

This function will be automatically tested on multiple Sudoku puzzles of increasing difficulty.

In [2]:
import numpy as np

def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input:
        sudoku : 9x9 numpy array
            Empty cells are represented by 0.

    Output:
        9x9 numpy array of integers
            Contains the solution, if one exists. If there is no solution, all array entries will be -1.
    """

    # Create a copy to avoid modifying the input
    board = sudoku.copy()

    # Check if the Sudoku is already invalid before attempting to solve
    if not is_initially_valid(board):
        return np.full((9, 9), -1)  # Return -1s because the puzzle is incorrect from the start

    # Check if the puzzle is solvable before solving
    if not is_solvable(board):
        return np.full((9, 9), -1)  # If unsolvable, return -1s

    # Solve the Sudoku using backtracking
    if solve_sudoku(board):
        return np.array(board)  # Convert to NumPy array before returning
    else:
        return np.full((9, 9), -1)  # Return -1s if no solution was found


def is_initially_valid(board):
    """Checks whether the initial Sudoku configuration is valid (no rule violations)."""
    for row in range(9):
        for col in range(9):
            num = board[row][col]
            if num != 0:  # If the cell already has a number, verify its validity
                board[row][col] = 0  # Temporarily remove it for validation
                if not is_valid(board, row, col, num):
                    return False  # If invalid, the Sudoku is already incorrect
                board[row][col] = num  # Restore the original number
    return True


def is_valid(board, row, col, num):
    """Checks if the number `num` can be placed at `board[row][col]` without breaking Sudoku rules."""
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False

    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False

    return True


def is_solvable(board):
    """Checks whether the Sudoku is solvable by ensuring that each empty cell has at least one valid option."""
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                possible_numbers = [num for num in range(1, 10) if is_valid(board, row, col, num)]
                if not possible_numbers:  # If no options for a cell, the puzzle is unsolvable
                    return False
    return True


def solve_sudoku(board):
    """Solves the Sudoku using backtracking."""
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                for num in range(1, 10):
                    if is_valid(board, row, col, num):
                        board[row][col] = num
                        if solve_sudoku(board):
                            return True
                        board[row][col] = 0  # Backtracking
                return False
    return True




## Testing and Evaluation

After implementing the Sudoku solver, we can evaluate how well it performs using a collection of puzzles categorized by difficulty: **very easy**, **easy**, **medium**, and **hard**. Each category contains 15 puzzles along with their correct solutions.

These categories are based on reference solvers, but depending on your algorithm, some puzzles might seem easier or harder than expected.

### Handling Invalid Puzzles

From the **easy** category onwards, some puzzles may contain invalid initial states, meaning they have no possible solution.  
In these cases, your function should return a 9x9 NumPy array filled entirely with the value `-1`.

### Performance Guidelines

Your solution will be tested in stages:

- First, it will be evaluated on the same **very easy** puzzles provided in the dataset.
- Then it will be tested on **additional hidden** puzzles from each difficulty level.
- A correct solution must exactly match the expected result.
- Puzzles must be solved in **under 30 seconds each**, although an average of **less than 1 second per puzzle** is recommended.

For difficult puzzles, correctness and speed both affect the final result. The best way to improve performance is by selecting efficient AI algorithms. This task is not designed to test parallelization or low-level optimizations, but rather your understanding of algorithmic problem solving.

### Running the Test Cell

You can run the test cell below to check your implementation. To activate the tests, change the value of the `SKIP_TESTS` constant to `False`.

Remember to change it back to `True` before submitting the notebook.



In [3]:
SKIP_TESTS = True

if not SKIP_TESTS:
    import time
    import numpy as np
    __SCORES = {}
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            if not isinstance(your_solution, np.ndarray):
                print("\033[91m[ERROR] Your sudoku_solver function returned a variable that has the incorrect type. If you submit this it will likely fail the auto-marking procedure result in a mark of 0 as it is expecting the function to return a numpy array with a shape (9,9).\n\t\033[94mYour function returns a {} object when {} was expected.\n\x1b[m".format(type(your_solution), np.ndarray))
            elif not np.all(your_solution.shape == (9, 9)):
                print("\033[91m[ERROR] Your sudoku_solver function returned an array that has the incorrect shape.  If you submit this it will likely fail the auto-marking procedure result in a mark of 0 as it is expecting the function to return a numpy array with a shape (9,9).\n\t\033[94mYour function returns an array with shape {} when {} was expected.\n\x1b[m".format(your_solution.shape, (9, 9)))
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took {} seconds to solve.\n".format(end_time-start_time))

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        __SCORES[difficulty] = {
            'correct': count,
            'total': len(sudokus)
        }

## Final Check Before Submission
The cell below runs a series of checks to confirm that the notebook is ready for submission.

To use it properly:

Restart the kernel and run all cells in order (use Kernel → Restart & Run All).

Then review the output of the submission test cell to ensure everything is correct.

In [4]:
import sys
import pathlib

fail = False;

success = '\033[1;32m[✓]\033[0m'
issue = '\033[1;33m[!]'
error = '\033[1;31m\t✗'
indent_success = '\033[1;32m\t✓'

#######
##
## Skip Tests check.
##
## Test to ensure the SKIP_TESTS variable is set to True to prevent it slowing down the automarker.
##
#######

if not SKIP_TESTS:
    fail = True;
    print("{} \'SKIP_TESTS\' is incorrectly set to False.\033[0m".format(issue))
    print("{} You must set the SKIP_TESTS constant to True in the cell above.\033[0m".format(error))
else:
    print('{} \'SKIP_TESTS\' is set to true.\033[0m'.format(success))

#######
##
## Report File Check.
##
## Test that checks there is a report pdf file found in the same folder as the notebook. This is required by the coursework specification.
##
#######

p1 = pathlib.Path('./report.pdf')
p2 = pathlib.Path('./Report.pdf')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("{} Report PDF not found.\033[0m".format(issue))
    print("{} You must include a separate file called report.pdf in your submission.\033[0m".format(error))
else:
    print('{} Report PDF found.\033[0m'.format(success))

#######
##
## File Name check.
##
## Test to ensure file has the correct name. This is important for the marking system to correctly process the submission.
##
#######
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("{} The notebook name is incorrect.\033[0m".format(issue))
    print("{} This notebook file must be named sudoku.ipynb\033[0m".format(error))
else:
    print('{} The notebook name is correct.\033[0m'.format(success))

#######
##
## Create classifier function check.
##
## Test that checks the create_classifier function exists. The function should train the classifier and return it so that it can be evaluated by the marking system.
##
#######

if "sudoku_solver" not in dir():
    fail = True;
    print("{} The sudoku_solver function has not been defined.\033[0m".format(issue))
    print("{} Your code must include a sudoku_solver function as described in the coursework specification.\033[0m".format(error))
    print("{} If you believe you have, \'restart & run-all\' to clear this error.\033[0m".format(error))
else:
    print('{} The sudoku_solver function has been defined.\033[0m'.format(success))



try:
    _sudoku = np.load("data/very_easy_puzzle.npy")[0]
    _solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(_sudoku), _solution):
        print("{} Your sudoku_solver function does not correctly solve the first sudoku.\033[0m".format(issue))
        print()
        print("{} Your assignment is unlikely to get any marks from the autograder. While we will\033[0m".format(error))
        print("{} try to check it manually to assign some partial credit, we encourage you to ask\033[0m".format(error))
        print("{} for help on the forum or directly to a tutor.\033[0m".format(error))
        print()
        print("{} Please use the report file to explain your code anyway.\033[0m".format(error))
    else:
        print("{} Your sudoku_solver function correctly solves the first sudoku.\033[0m".format(success))
        if "__SCORES" in dir():
#             print("{} Test set summary - Not Found.\033[0m".format(issue))
#             print("{} Test set summary could not be found. This is automatically generated when the \033[0m".format(error))
#             print("{} above test cell is run. If you would like to see the summary please run the above cell.\033[0m".format(error))
#             print("{} You do not need this for submission, it is only for your convenience.\033[0m".format(error))
#         else:
            correct = 0
            total = 0
            for key, value in __SCORES.items():
                correct += value['correct']
                total += value['total']
                
            print("{} Test set summary - {}/{} Correct.\033[0m".format(issue, correct, total))
            if total != correct:
                
                for key, value in __SCORES.items():
                    if value['correct'] == value['total']:
                        print("{} {}/{} of {} sudokus correct.\033[0m".format(indent_success, value['correct'], value['total'], key))
                    else:
                        print("{} {}/{} of {} sudokus correct.\033[0m".format(error, value['correct'], value['total'], key))
            
except Exception as e:
    fail = True
    print("{} Error running test set.\033[0m".format(issue))
    print("{} Your code produced the following error. This error will result in a zero from the automarker, please fix.\033[0m".format(error))
    print(e)

    

#######
##
## Final Summary
##
## Prints the final results of the submission tests.
##
#######

if fail:
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("\033[1m\n\n")
    print("╔═══════════════════════════════════════════════════════════════╗")
    print("║                        Congratulations!                       ║")
    print("║                                                               ║")
    print("║            Your work meets all the required criteria          ║")
    print("║                   and is ready for submission.                ║")
    print("╚═══════════════════════════════════════════════════════════════╝")
    print("\033[0m")
    

[✓] 'SKIP_TESTS' is set to true.
[✓] Report PDF found.
[✓] The notebook name is correct.
[✓] The sudoku_solver function has been defined.
[✓] Your sudoku_solver function correctly solves the first sudoku.



╔═══════════════════════════════════════════════════════════════╗
║                        Congratulations!                       ║
║                                                               ║
║            Your work meets all the required criteria          ║
║                   and is ready for submission.                ║
╚═══════════════════════════════════════════════════════════════╝



In [5]:
# This is a TEST CELL. Do not delete or change.